In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]
DATA_PATH = "../../.data"
CREDENTIALS_JSON_PATH = f"{DATA_PATH}/credentials.json"
TOKEN_JSON_PATH = f"{DATA_PATH}/token.json"

In [ ]:
def authenticate_gmail_api():
    """Authenticates with the Gmail API, handling refresh tokens.

    This function attempts to load existing user credentials from 'token.json'.
    If credentials are not found or are invalid/expired, it initiates a new
    OAuth 2.0 flow to prompt the user for authorization in their browser.
    The 'refresh_token' is automatically handled by InstalledAppFlow when
    'access_type=offline' is implicitly or explicitly requested, allowing
    the application to obtain new access tokens without re-prompting the user.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(TOKEN_JSON_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_JSON_PATH, SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print("Access token expired, attempting to refresh...")
            creds.refresh(Request())
        else:
            print("No valid credentials found, initiating new OAuth flow...")
            # Use InstalledAppFlow for desktop applications.
            # The 'credentials.json' file is the one downloaded from Google Cloud.
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_JSON_PATH, SCOPES
            )
            # Run the local server to handle the OAuth 2.0 redirect.
            # This will open a browser window for the user to grant permissions.
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run, including the refresh token.
        with open(TOKEN_JSON_PATH, "w") as token:
            token.write(creds.to_json())

    # Build and return the Gmail API service object.
    try:
        service = build("gmail", "v1", credentials=creds)
        print("Gmail API service successfully built.")
        return service
    except HttpError as error:
        print(f"An error occurred while building the Gmail API service: {error}")
        return None

In [ ]:
def get_messages_list(gmail_service, userId: str = "me", maxResults: int = 5):
    if gmail_service:
        # Now you can use the 'gmail_service' object to interact with the Gmail API.
        # For instance, to list messages:
        try:
            results = (
                gmail_service.users()
                .messages()
                .list(
                    userId=userId,
                    maxResults=maxResults,
                    q="has:attachment filename:pdf filename:xml subject:factura",
                )
                .execute()
            )
            messages = results.get("messages", [])

            return messages
        except HttpError as error:
            print(f"An error occurred while listing messages: {error}")

In [ ]:
def get_message_details(
    gmail_service,
    id: str,
    userId: str = "me",
):
    if gmail_service:
        # Now you can use the 'gmail_service' object to interact with the Gmail API.
        # For instance, to list messages:
        try:
            result = (
                gmail_service.users()
                .messages()
                .get(userId=userId, id=id, format="minimal")
                .execute()
            )

            return result
        except HttpError as error:
            print(f"An error occurred while listing messages: {error}")

In [ ]:
gmail_service = authenticate_gmail_api()

messages = get_messages_list(gmail_service)

messagesDetails = {}

if not messages:
    print("No messages found.")
else:
    print("First 5 message IDs:")
    for message in messages:
        messageId = message["id"]
        print(f"Fetching details for message {messageId}")
        messageDetails = get_message_details(gmail_service, messageId)
        print(messageDetails)

        messagesDetails[messageId] = messageDetails

In [ ]:
messagesDetails["19778f3866746ced"]